In [18]:
## DO NOT USE -  Old way using PDFMiner. Problem was every file had a different format. 
"""
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt
import codecs
import pandas as pd
import numpy as np

def convert_to_csv(fname):
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname+'.pdf', 'rb')
    for page in PDFPage.get_pages(infile, set()):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    f = open(fname+'.csv', "w") #make text file
    f.write(text) #write text to text file
    output.close

convert_to_csv('/Users/purnimakamath/appdir/Coursera/Applied Data Science/Week4 - Assignment/MOH-Data/2005_week_52')

def extract_data(fname, year):
    indices = {
        2005:{'m_i':122,'d_i':120,'af_i':9999,'s_i':131},
        2006:{'m_i':88,'d_i':86,'af_i':9999,'s_i':97},
        2007:{'m_i':134,'d_i':132,'af_i':9999,'s_i':143},
        2008:{'m_i':86,'d_i':84,'af_i':9999,'s_i':94},
        2009:{'m_i':67,'d_i':65,'af_i':126,'s_i':139},
        2010:{'m_i':89,'d_i':87,'af_i':237,'s_i':250},
        2011:{'m_i':95,'d_i':93,'af_i':100,'s_i':113}
    }
    #print(indices[2005]['m_i'])
    with open(fname) as f:
        data = list(f)
    malaria = int(data[indices[year]['m_i']].split('   ')[0])
    dengue = int(data[indices[year]['d_i']].split('   ')[0])
    avian_flu = 0
    if indices[year]['af_i'] != 9999:
        avian_flu = int(data[indices[year]['af_i']].split('   ')[0])
    sars = int(data[indices[year]['s_i']].split('   ')[0])
    f.close()
    return (malaria, dengue, avian_flu, sars)

extract_data('/Users/purnimakamath/appdir/Coursera/Applied Data Science/Week4 - Assignment/MOH-Data/2005_week_52.csv',2005)

years=range(2005,2012,1)

for year in list(years):
    for i in range(1,53,1):
        file_name = str(year)+"_week_"+str(i)
        file_location = "/Users/purnimakamath/appdir/Coursera/Applied Data Science/Week4 - Assignment/MOH-Data/"+file_name
        convert_to_csv(file_location)
        extract_data(file_location+".csv", year)
"""       
## DO NOT USE - Old way completed!

In [263]:
# Converted PDFs using MAC OS Automator - http://osxdaily.com/2016/11/05/convert-pdf-to-docx-mac-os-x/ (option 4)
import os
import sys, getopt
import codecs
import pandas as pd
import numpy as np

txt_location = 'data/MOH-Data-TXT/'
years=range(2004,2012,1)

yr, week, dengue, malaria, avianflu, sars, chikunguniya = (list(), list(),list(),list(),list(),list(),list())
for year in list(years):
    for i in range(1,53,1):
        data = list()
        if (((year == 2004) & (i >= 5)) | (year != 2004)):
            file_name = str(year)+"_week_"+str(i)
            file_location = txt_location+file_name+'.txt'
            try:
                with codecs.open(file_location, "r",encoding='utf-8', errors='ignore') as f:
                    data = list(f)
            except FileNotFoundError:
                file_name = str(year)+"_week_0"+str(i)
                file_location = txt_location+file_name+'.txt'
                with codecs.open(file_location, "r",encoding='utf-8', errors='ignore') as f1:
                    data = list(f1)
                    
            d_i, m_i, af_i, s_i, c_i = (0,0,9999,0,9999)
            for idx, l in enumerate(data):
                if (('Dengue Fever' in l.strip()) & (d_i == 0)):
                    d_i = idx
                if (('Malaria' in l.strip()) & (m_i == 0)):
                    m_i = idx
                if (('Severe acute respiratory syndrome' in l.strip()) & (s_i == 0)):
                    s_i = idx
                if (('Avian Influenza' in l.strip()) & (af_i == 9999)):
                    af_i = idx
                if (('Chikungunya Fever' in l.strip()) & (c_i == 9999)):
                    c_i = idx
            
            af_val = 0
            if af_i != 9999:
                af_val = int(data[af_i+1].replace('\n',''))
            c_val = 0
            if (c_i != 9999) & (year != 2008):
                c_val = int(data[c_i+1].replace('\n',''))
            d_val = int(data[d_i+1].replace('\n',''))
            s_val = int(data[s_i+1].replace('\n',''))
            m_val = int(data[m_i+1].replace('\n',''))
            yr.append(year)
            week.append(i)
            dengue.append(d_val)
            malaria.append(m_val)
            avianflu.append(af_val)
            sars.append(s_val)
            chikunguniya.append(c_val)

disease_df = pd.DataFrame()
disease_df['year'] = yr
disease_df['week'] = week
disease_df['dengue'] = dengue
disease_df['malaria'] = malaria
disease_df['avianflu'] = avianflu
disease_df['sars'] = sars
disease_df['chikunguniya'] = chikunguniya

#disease_df[disease_df['year'] == 2004]

In [264]:
# Read data for 2001 to 2004 May and append to the main dataframe

disease_manual_df = pd.read_csv('data/manually-2001-to-2004.csv', usecols=['year','week','dengue','malaria','avianflu','sars','chikunguniya'])

disease_df = disease_manual_df.append(disease_df)
disease_df = disease_df.fillna(0)
disease_df = disease_df.drop(['week'], axis=1)
disease_df.head()

disease_df = disease_df.groupby(by='year').sum()
disease_df



,dengue,malaria,avianflu,sars,chikunguniya
year,,,,,
2001,663,266,0,0,0
2002,3938,175,0,0,0
2003,4732,115,0,239,0
2004,9302,153,0,0,0
2005,13797,155,0,0,0
2006,3058,189,0,0,0
2007,8664,159,0,0,0
2008,6768,159,0,0,0
2009,4480,172,0,0,375


In [265]:
# Read data for 2012 to 2016 and append to the main data frame

disease_2012_onwards_df = pd.read_csv('data/weekly-infectious-disease-bulletin-cases/weekly-infectious-disease-bulletin-cases.csv')

disease_2012_onwards_df['year'] = disease_2012_onwards_df['epi_week'].apply(lambda x: x.split('-')[0])
disease_2012_onwards_df = disease_2012_onwards_df.where((disease_2012_onwards_df['year'] != '2017')
                                                        & ((disease_2012_onwards_df['disease'] == 'Avian Influenza')
                                                       | (disease_2012_onwards_df['disease'] == 'Chikungunya Fever')
                                                       | (disease_2012_onwards_df['disease'] == 'Dengue Fever')
                                                       | (disease_2012_onwards_df['disease'] == 'Malaria')
                                                       | (disease_2012_onwards_df['disease'] == 'SARS'))).dropna()
disease_2012_onwards_df_u = pd.DataFrame()
disease_2012_onwards_df_u['year'] = disease_2012_onwards_df['year'].unique()
disease_2012_onwards_df_u['dengue'] = disease_2012_onwards_df.where(disease_2012_onwards_df['disease'] == 'Dengue Fever').dropna().groupby(by='year')['no._of_cases'].sum().values
disease_2012_onwards_df_u['malaria'] = disease_2012_onwards_df.where(disease_2012_onwards_df['disease'] == 'Malaria').dropna().groupby(by='year')['no._of_cases'].sum().values
disease_2012_onwards_df_u['avianflu'] = disease_2012_onwards_df.where(disease_2012_onwards_df['disease'] == 'Avian Influenza').dropna().groupby(by='year')['no._of_cases'].sum().values
disease_2012_onwards_df_u['sars'] = disease_2012_onwards_df.where(disease_2012_onwards_df['disease'] == 'SARS').dropna().groupby(by='year')['no._of_cases'].sum().values
disease_2012_onwards_df_u['chikunguniya'] = disease_2012_onwards_df.where(disease_2012_onwards_df['disease'] == 'Chikungunya Fever').dropna().groupby(by='year')['no._of_cases'].sum().values

disease_2012_onwards_df_u = disease_2012_onwards_df_u.set_index('year')

disease_2012_onwards_df_u



,dengue,malaria,avianflu,sars,chikunguniya
year,,,,,
2012,4602.0,143.0,0.0,0.0,22.0
2013,22101.0,111.0,0.0,0.0,1059.0
2014,18306.0,62.0,0.0,0.0,185.0
2015,11291.0,49.0,0.0,0.0,43.0
2016,13153.0,30.0,1.0,0.0,39.0


In [266]:
disease_df = disease_df.append(disease_2012_onwards_df_u)

disease_df

,dengue,malaria,avianflu,sars,chikunguniya
year,,,,,
2001,663.0,266.0,0.0,0.0,0.0
2002,3938.0,175.0,0.0,0.0,0.0
2003,4732.0,115.0,0.0,239.0,0.0
2004,9302.0,153.0,0.0,0.0,0.0
2005,13797.0,155.0,0.0,0.0,0.0
2006,3058.0,189.0,0.0,0.0,0.0
2007,8664.0,159.0,0.0,0.0,0.0
2008,6768.0,159.0,0.0,0.0,0.0
2009,4480.0,172.0,0.0,0.0,375.0


In [267]:
disease_df.to_csv('data/final-disease.csv')